# Выбор региона для миграции семьи

![title](https://raw.githubusercontent.com/vyushmanov/ML-Analitics/master/2020-08-26_Region_family_matrix/Pic.JPG)

In [1]:
# БИБЛИОТЕКИ

import re
import numpy as np
import pandas as pd

from tabulate import tabulate 
from IPython.display import HTML, display
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_rows', 50) # выведем больше строк
pd.set_option('display.max_columns', 50) # выведем больше колонок
import warnings; warnings.simplefilter('ignore') #  отключение вывода предупреждающих сообщений


# ФУНКЦИИ

def standardization_name(df, original_name, small_name):
    pattern = '[Городг.Республика]*\s*([\w\-]+)'
    df[small_name] = df[original_name].apply(lambda x: re.findall(pattern, x.strip())[0])
    return df
    
def show_matrix(matrix, target_columns):
    matrix = matrix[target_columns]
    matrix = matrix.fillna(0)
    for i in target_columns[2:]:
        matrix[i] = matrix[i].map(lambda x: 0 if type(x) == 'str' else x)
        
        if 'delta' in i:
            matrix[i] = matrix[i].map(lambda x: '' if x == 0 else '+' + str(int(x)) + '%')
        elif 'share' in i or 'grow' in i or 'scre' in i or i == 'Код':
            matrix[i] = matrix[i].map(lambda x: '' if x == 0 else str(round(x,1)) + '%')
        elif 'scho' in i:
            matrix[i] = matrix[i].map(lambda x: '' if x == 0 else str(round(x,3)))
        else:
            matrix[i] = matrix[i].map(lambda x: '' if x == 0 else str(int(x)) if x > 1000 else x)
    
    return matrix 


# Код, позволяющий скрывать ячейки
from IPython.display import HTML
import random

def hide_toggle(toggle_text = 'Toggle show/hide', for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

#    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle('Блок функций и библиотек')


### Мероприятия по подготовке данных к исследованию

При работе со статистическими данными, касающимися регионов и субъектов федерации, приходится решать проблему несоответствия наименований объектов, используемых в различных источниках.

В целях стандартизации перечня наименований произведем его приведение к Списоку регионов России в порядке заполнения формы РСВ (Приложение № 7).
Для этого сократим используемые наименования устранив упоминание о форме административного образования (область, край и т.д.)


In [2]:
# Стандартизация наименований субъектов федерации

xls = pd.ExcelFile('codeSF.xlsx')
sf_code = pd.read_excel(xls, 'Sheet1')
sf_code.dropna(axis = 0, inplace = True)

sf_code = standardization_name(sf_code, 'Наименование', 'sm_name')

## Доходы и расходы населения



In [3]:
# Смотрим на доходы

xls = pd.ExcelFile('Revenue.xlsx')
revenue = pd.read_excel(xls, '2013-2018',header = 3)
revenue = revenue.dropna(how = 'all', axis = 0, thresh = 30)

revenue19 = pd.read_excel(xls, '2019',header = 2)
revenue19 = revenue19.dropna(how = 'all', axis = 0, thresh = 11)
revenue19 = revenue19.dropna(axis = 1)
revenue = pd.merge(revenue, revenue19, on = ['Unnamed: 0'])
revenue = pd.DataFrame(np.rot90(np.array(revenue).T, 3))
revenue = revenue[revenue.columns.to_list()[::-1]]
column_list = ['name']
m = 1
y = 2013
for i in revenue.columns.to_list()[1:]:
    column_list.append(str(y) + '-' + str(m).zfill(2))
    m += 1
    if m == 13:
        m = 1
        y += 1
revenue.columns = column_list

matrix = pd.DataFrame(revenue['name'])

for y in range(2013, 2019+1):
    temp_column_list = ['name']
    for c in column_list:
        if str(y) in c:
            temp_column_list.append(c)
    df = revenue[temp_column_list]
    res = []
    for s in np.array(df):
        res.append(round(np.median(s[1:]), 2))
    matrix[str(y)] = res
matrix['revenue17'] = matrix['2017'].apply(lambda x: x*2)
matrix['revenue19'] = matrix['2019'].apply(lambda x: x*2)
revenue = standardization_name(matrix, 'name', 'sm_name')
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 11 columns):
name         96 non-null object
2013         94 non-null float64
2014         94 non-null float64
2015         96 non-null float64
2016         96 non-null float64
2017         96 non-null float64
2018         96 non-null float64
2019         96 non-null float64
revenue17    96 non-null float64
revenue19    96 non-null float64
sm_name      96 non-null object
dtypes: float64(9), object(2)
memory usage: 8.4+ KB


In [4]:
# Исследуем данные о прожиточном минимуме

xls = pd.ExcelFile('Coasts.xls')
coast17 = pd.read_excel(xls, 'Данные',header = 4)
coast17 = coast17.dropna(how = 'all', axis = 0, thresh = 5)
coast17.rename(columns = lambda x: x.replace('Unnamed: 0', 'name'), inplace=True)
coast17['chil'] = coast17.apply(lambda x: [x['Дети'], x['Дети.1'], x['Дети.2'], x['Дети.2']], axis = 1)
coast17['chil'] = coast17['chil'].map(lambda x: np.median(x))
coast17['work'] = coast17.apply(lambda x: [x['Трудоспособное население'], x['Трудоспособное население.1'],
                                         x['Трудоспособное население.2'], x['Трудоспособное население.2']], axis = 1)
coast17['work'] = coast17['work'].map(lambda x: np.median(x))
coast17['coast17'] = coast17.apply(lambda x: x['work']*2 + x['chil']*2, axis = 1)
coast17['name'] = coast17['name'].apply(lambda x: x.replace('федерального значения ',''))
coast17 = coast17[['name', 'coast17']]
coast17 = standardization_name(coast17, 'name', 'sm_name')

xls = pd.ExcelFile('vpm.xlsx')
coast19 = pd.read_excel(xls, 'Sheet1',header = 2)
coast19.rename(columns = lambda x: x.replace('Unnamed: 0', 'name'), inplace=True)
coast19 = coast19.dropna(how = 'all', axis = 0, thresh = 3)
coast19['coast19'] = coast19.apply(lambda x: x['трудоспособное население']*2 + x['дети']*2, axis = 1)
coast19['name'] = coast19['name'].apply(lambda x: x.replace('авт.область','автономная область')
                                                    .replace('авт.округ','автономный округ'))
coast19 = coast19[['name', 'coast19']]
coast19 = standardization_name(coast19, 'name', 'sm_name')

coasts = pd.merge(sf_code, coast17, how = 'left', on = ['sm_name'])
coasts = pd.merge(coasts, coast19, how = 'left', on = ['sm_name'])
columns = ['Код', 'coast17', 'coast19', 'sm_name']
coasts = coasts[columns]

coasts['coast17'] = coasts['coast17'].apply(lambda x: None if x == 0 else x)
coasts['coast19'] = coasts['coast19'].apply(lambda x: None if x == 0 or '-' in str(x) else x)
coasts.dropna(axis = 0, inplace = True)
coasts['delta_coast'] = coasts.apply(lambda x: round((x['coast19'] - x['coast17']) / x['coast17'] * 100, 2), axis = 1)
coasts = coasts.sort_values(by = ['coast19'], ascending = False)
coasts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 87 to 33
Data columns (total 5 columns):
Код            83 non-null float64
coast17        83 non-null float64
coast19        83 non-null float64
sm_name        83 non-null object
delta_coast    83 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.9+ KB


In [5]:
# Формируем сравнение покупательской способности по регионам

matrix = pd.merge(revenue, coasts, how = 'left', on = ['sm_name'])
#matrix.drop(['Код', 'sm_name'], axis=1, inplace = True)
matrix['real_revenue17'] = matrix.apply(lambda x: x['revenue17'] / x['coast17'], axis = 1)
matrix['real_revenue19'] = matrix.apply(lambda x: round(x['revenue19'] / x['coast19'], 2), axis = 1)

matrix['delta_real_rev'] = matrix.apply(lambda x: round((x['real_revenue19'] - x['real_revenue17']) / x['real_revenue17'] * 100, 2), axis = 1)
matrix = matrix.sort_values(by = ['real_revenue19'], ascending = False)
matrix['welfare_rank'] = range(1, len(matrix)+1)
matrix = matrix.sort_index()
view_matrix = pd.DataFrame(matrix) 
view_matrix['delta_real_rev%'] = view_matrix['delta_real_rev'].map(lambda x: '+' + str(x) + '%' if x >= 0 else str(x) + '%')

key_columns = ['Код', 'name', 'revenue19', 'coast19', 'real_revenue19', 'delta_real_rev%', 'welfare_rank']
view_matrix = view_matrix.sort_values(by = ['real_revenue19'], ascending = False)
view = view_matrix[key_columns].dropna(axis = 0)
display(HTML(tabulate(np.array(view[view['real_revenue19'] > 1.8]), headers = key_columns, tablefmt='html', showindex="always")))
matrix.info(verbose = False)

,Код,name,revenue19,coast19,real_revenue19,delta_real_rev%,welfare_rank
0,72,Тюменская область,138690,47402,2.93,+6.01%,1
1,89,Ямало-Ненецкий авт. округ,192782,68814,2.8,+6.54%,2
2,65,Сахалинская область,169186,63832,2.65,+13.9%,3
3,77,г.Москва,178090,67996,2.62,+20.66%,4
4,78,г.Санкт-Петербург,122282,47520,2.57,+7.28%,5
5,87,Чукотский авт.округ,208717,92708,2.25,+1.39%,6
6,49,Магаданская область,176085,83752,2.1,+8.27%,7
7,50,Московская область,108197,51924,2.08,+13.92%,8
8,47,Ленинградская область,90737,44050,2.06,+3.94%,9
9,14,Республика Саха (Якутия),139651,70828,1.97,+14.92%,10


<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Columns: 20 entries, name to delta_real_rev%
dtypes: float64(16), int32(1), object(3)
memory usage: 15.4+ KB


В представленной таблице:
* __name__ - наименование субъекта федерации
* __revenue19__ - средний доход семьи с двумя работающими родителями, 2019 год
* __coast19__ - прожиточный минимум для семьи, состоящей из двух взрослых и двух детей, 2019 г.
* __real_revenue19__ - покупательская способность семьи, рассчитанная как частное средних доходов семьи к ее среднему прожиточному минимуму, 2019 г.
* __delta_real_rev__ - изменение покупательской способности семьи, произошедшее в период с 2017 по 2019 гг.

Таблица отсортирована и отфильтрована по показателю покупательской способности.

In [6]:
# Произведем оценку численности населения по состояния на 1 января 2019 года.

xls = pd.ExcelFile('Population.xlsx')
people = pd.read_excel(xls, 'Sheet1',header = 2)
people.rename(columns = lambda x: x.replace('Unnamed: 0', 'name').replace('Все', 'people'), inplace=True)
people = standardization_name(people[['name', 'people']], 'name', 'sm_name')
matrix = pd.merge(matrix, people.drop(['name'], axis = 1), how = 'left', on = ['sm_name'])
matrix = matrix.drop_duplicates(subset = ['sm_name', 'real_revenue19'], keep='first')

print('ДОБАВЛЕНЫ СТОЛБЦЫ:')
people.drop(['name', 'sm_name'], axis = 1).info()
print('\nВ МАТРИЦЕ:')
matrix.info(verbose = False)

ДОБАВЛЕНЫ СТОЛБЦЫ:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 1 columns):
people    96 non-null int64
dtypes: int64(1)
memory usage: 896.0 bytes

В МАТРИЦЕ:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 99
Columns: 21 entries, name to people
dtypes: float64(17), int32(1), object(3)
memory usage: 16.1+ KB


In [7]:
# Разберем статистику ВУЗов

xls = pd.ExcelFile('High_scool.xls')
high_school = pd.read_excel(xls, 'Таблица 1', header = 6)
high_school = high_school.dropna(how = 'all', axis = 0, thresh = 15)

high_school.rename(columns = lambda x: str(x).replace('1', 'name')
                   .replace('2', 'school')
                   .replace('6', 'school_f')
                   .replace('7', 'students') if len(str(x)) == 1 else x, inplace=True)
new_column = []
for i in high_school.columns.to_list():
    new_column.append(str(i))
high_school.columns = new_column   

high_school = standardization_name(high_school, 'name', 'sm_name')
high_school = pd.merge(high_school, people.drop(['name'], axis = 1), how = 'left', on = ['sm_name'])

high_school['school_on_10^5_people']     = high_school.apply(lambda x: (x['school'] + x['school_f']) / x['people'] * 100000, axis = 1)
high_school['share_stud_people']  = high_school.apply(lambda x: x['students'] / x['people'] * 100, axis = 1)
high_school['share_full_stud']    = high_school.apply(lambda x: x['8'] / x['students'] * 100 if x['students'] != 0 else None, axis = 1)
high_school['share_evening_stud'] = high_school.apply(lambda x: x['9'] / x['students'] * 100 if x['students'] != 0 else None, axis = 1)
high_school['share_corr_stud']    = high_school.apply(lambda x: x['10'] / x['students'] * 100 if x['students'] != 0 else None, axis = 1)
high_school.rename(columns = lambda x: str(x).replace('13', 'share_stud_femail'), inplace=True)
high_school['growth_stud']        = high_school.apply(lambda x: (x['16'] - x['20']) / x['20'] * 100 if x['20'] != 0 else None, axis = 1)
high_school['screening_out_stud'] = high_school.apply(lambda x: (x['16'] - x['20']) / x['16'] * 100 if x['16'] != 0 else None, axis = 1)
high_school['share_extern_stud']  = high_school.apply(lambda x: x['24'] / x['20'] * 100 if x['20'] != 0 else None, axis = 1)
high_school = high_school[['name', 'sm_name', 'school_on_10^5_people', 'students', 'share_stud_people'
                           , 'share_full_stud', 'share_evening_stud', 'share_corr_stud', 'share_stud_femail'
                           , 'growth_stud', 'screening_out_stud', 'share_extern_stud']]

matrix = pd.merge(matrix, high_school.drop(['name'], axis = 1), how = 'left', on = ['sm_name'])
matrix = matrix.drop_duplicates(subset = ['name', 'real_revenue19'], keep='first')

print('ДОБАВЛЕНЫ СТОЛБЦЫ:')
high_school.drop(['name', 'sm_name'], axis = 1).info()
print('\nВ МАТРИЦЕ:')
matrix.info(verbose = False)

ДОБАВЛЕНЫ СТОЛБЦЫ:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 10 columns):
school_on_10^5_people    95 non-null float64
students                 96 non-null int64
share_stud_people        95 non-null float64
share_full_stud          95 non-null float64
share_evening_stud       95 non-null float64
share_corr_stud          95 non-null float64
share_stud_femail        96 non-null float64
growth_stud              95 non-null float64
screening_out_stud       95 non-null float64
share_extern_stud        95 non-null float64
dtypes: float64(9), int64(1)
memory usage: 8.2 KB

В МАТРИЦЕ:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 99
Columns: 31 entries, name to share_extern_stud
dtypes: float64(27), int32(1), object(3)
memory usage: 23.6+ KB


In [8]:
# Проведем оценку численности преподавательского состава
xls = pd.ExcelFile('Professors.xls')
teachers = pd.read_excel(xls, 'Таблица 1', header = 6)

teachers.rename(columns = lambda x: str(x).replace('11', 'share_mag_teach')
                    .replace('1', 'name')
                    .replace('2', 'teachers')
                    .replace('9', 'share_doc_teach'), inplace=True)

teachers = standardization_name(teachers[['name', 'teachers', 'share_doc_teach', 'share_mag_teach']], 'name', 'sm_name')
teachers = pd.merge(teachers, matrix[['sm_name', 'students']], how = 'left', on = ['sm_name'])

teachers['share_teach_stud'] = teachers.apply(lambda x: x['teachers'] / x['students'] * 100, axis = 1)
teachers = teachers[['sm_name', 'teachers', 'share_teach_stud', 'share_doc_teach', 'share_mag_teach']]

# Добавим полученные данные к матрице
matrix = pd.merge(matrix, teachers, how = 'left', on = ['sm_name'])
matrix = matrix.drop_duplicates(subset = ['name', 'real_revenue19'], keep='first')

print('ДОБАВЛЕНЫ СТОЛБЦЫ:')
teachers.drop(['sm_name'], axis = 1).info()
print('\nВ МАТРИЦЕ:')
matrix.info(verbose = False)

ДОБАВЛЕНЫ СТОЛБЦЫ:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 4 columns):
teachers            96 non-null int64
share_teach_stud    95 non-null float64
share_doc_teach     96 non-null float64
share_mag_teach     96 non-null float64
dtypes: float64(3), int64(1)
memory usage: 3.8 KB

В МАТРИЦЕ:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 99
Columns: 35 entries, name to share_mag_teach
dtypes: float64(31), int32(1), object(3)
memory usage: 26.6+ KB


In [9]:
# Средний возраст преподавателей

xls = pd.ExcelFile('Professors_age.xls')
teachers_age = pd.read_excel(xls, 'Таблица 1', header = 5)

vector_age = np.array([22,25,30,35,40,45,50,55,60,65,70])

teachers_age.rename(columns = lambda x: str(x).replace('1', 'name'), inplace=True)
teachers_age = standardization_name(teachers_age, 'name', 'sm_name')
teachers_age = pd.merge(matrix[['sm_name']], teachers_age, how = 'left', on = ['sm_name'])
teachers_age = teachers_age.fillna(0)

teachers_age.drop(['name', 'sm_name'], inplace = True, axis = 1)
v_teachers_age = np.array(teachers_age)
v_teachers_age = np.around(v_teachers_age)

# Расчет среднего возраста преподавателей
matrix_age = v_teachers_age * vector_age[:-1]
avg_teach_age = []
for a in matrix_age:
    avg_teach_age.append(np.around(sum(a)/100, decimals = 1))
    
matrix['teacher_age_avg'] = avg_teach_age

# Рассчет медианных значений возраста преподавателей
period_age = (vector_age[1:] - vector_age[:-1]) / v_teachers_age
median_age = []
for indexS, s in enumerate(v_teachers_age):
    str_median_age = []
    for indexC, c in enumerate(s):
        for i in range(int(c)):
            str_median_age.append(vector_age[indexC] + i * period_age[indexS][indexC])
    median_age.append(np.median(str_median_age))
median_age = np.around(median_age, decimals = 1)

matrix['teacher_age_median'] = median_age
matrix = matrix.drop_duplicates(subset = ['name', 'real_revenue19'], keep='first')


print('ДОБАВЛЕНЫ СТОЛБЦЫ\nteacher_age_avg\nteacher_age_median \n\nВ МАТРИЦЕ:')
matrix.info(verbose = False)

ДОБАВЛЕНЫ СТОЛБЦЫ
teacher_age_avg
teacher_age_median 

В МАТРИЦЕ:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 99
Columns: 37 entries, name to teacher_age_median
dtypes: float64(33), int32(1), object(3)
memory usage: 28.1+ KB


In [10]:
# Вывод на экран итоговой матрицы параметров

target_columns = ['Код', 'name', 'welfare_rank', 'school_on_10^5_people'
                  , 'share_stud_people', 'share_full_stud'
                  , 'share_stud_femail', 'growth_stud', 'screening_out_stud', 'share_extern_stud' 
                  , 'share_teach_stud', 'share_doc_teach', 'share_mag_teach', 'teacher_age_median']

print('Для сортировки таблицы статистических данных установите приоритеты, выбрав варианты:\n')
a = input('Большое количество ВУЗов на 100000 человек - это:           0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - ')
b = input('Стремление населения к получению высшего образования - это: 0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - ')
c = input('Высокая доля студентов на дневной форме обучения - это:     0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - ')
d = input('Высокая доля женщин среди студентов - это:                  0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - ')
e = input('Высокий отсев неуспевающих - это:                          -1 - плохо; 0 - не важно; 1 - хорошо          | Ваш ответ - ')
f = input('Высокий процент экстернатуры - это:                         0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - ')
g = input('Высокая доля преподавателей, имеющих научную степень - это: 0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - ')
h = input('Молодой возраст преподавателей - это:                       0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - ')

print('\n')

priority_vector = [int(a),int(b),int(c),int(d),int(e),int(f),int(g),int(h)]

#priority_vector = [1,1,1,1,1,0,0,1]

def priority_distribution(view, vector):
    scaler = MinMaxScaler()
    
    view['a'] = scaler.fit_transform(np.array(view['school_on_10^5_people']).reshape(-1,1))
    view['b'] = scaler.fit_transform(np.array(view['share_stud_people']).reshape(-1,1))
    view['c'] = scaler.fit_transform(np.array(view['share_full_stud']).reshape(-1,1))
    view['d'] = scaler.fit_transform(np.array(view['share_stud_femail']).reshape(-1,1))
    view['e'] = scaler.fit_transform(np.array(view['screening_out_stud']).reshape(-1,1))
    view['f'] = scaler.fit_transform(np.array(view['share_extern_stud']).reshape(-1,1))
    view['g'] = scaler.fit_transform((np.array(view['share_doc_teach'])*2 + np.array(view['share_mag_teach']))
                                     .reshape(-1,1))
    view['h'] = scaler.fit_transform(1 - np.array(view['teacher_age_median']).reshape(-1,1))
                                     
    priority_matrix = np.array(view[['a','b','c','d','e','f','g','h']]) * vector
    priority_vector = []
    for c in priority_matrix:
        priority_vector.append(sum(c))
    view['rank'] = priority_vector
    view = view.sort_values(by = 'rank', ascending = False)
    return view

view = priority_distribution(matrix, priority_vector)
view = show_matrix(view, target_columns + ['rank'])
view = view[view['rank'] != '']
view = view[view['rank'] > view['rank'].quantile(0.9)]

display(HTML(tabulate(np.array(view[target_columns]), headers = target_columns, tablefmt='html', showindex="always")))

hide_toggle('Код ячейки скрыт')

Для сортировки таблицы статистических данных установите приоритеты, выбрав варианты:

Большое количество ВУЗов на 100000 человек - это:           0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - 1
Стремление населения к получению высшего образования - это: 0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - 2
Высокая доля студентов на дневной форме обучения - это:     0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - 1
Высокая доля женщин среди студентов - это:                  0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - 0
Высокий отсев неуспевающих - это:                          -1 - плохо; 0 - не важно; 1 - хорошо          | Ваш ответ - 0
Высокий процент экстернатуры - это:                         0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - 1
Высокая доля преподавателей, имеющих научную степень - это: 0 - не важно; 1 - важно; 2 - очень важно     | Ваш ответ - 2
Молодой возраст преподавателей - это:                       0 - не 

,Код,name,welfare_rank,school_on_10^5_people,share_stud_people,share_full_stud,share_stud_femail,growth_stud,screening_out_stud,share_extern_stud,share_teach_stud,share_doc_teach,share_mag_teach,teacher_age_median
0,70,Томская область,26,1.019,5.3%,66.0%,49.4%,48.0%,32.4%,,6.6%,19.1%,55.4%,48
1,57,Орловская область,73,1.227,3.9%,51.3%,54.8%,11.4%,10.2%,,4.9%,14.5%,68.5%,45.4
2,68,Тамбовская область,63,0.695,2.8%,50.2%,52.0%,27.6%,21.7%,1.0%,4.4%,18.2%,70.1%,44.4
3,16,Республика Татарстан,12,1.051,3.7%,60.6%,52.2%,29.4%,22.7%,,5.0%,15.5%,61.1%,46
4,77,г.Москва,4,1.199,5.5%,66.7%,52.7%,17.8%,15.1%,1.9%,6.8%,18.7%,52.8%,51.9
5,46,Курская область,52,0.996,3.3%,51.8%,57.3%,3.4%,3.3%,0.0%,4.8%,17.9%,63.6%,45.5
6,78,г.Санкт-Петербург,5,1.297,5.6%,72.6%,53.7%,44.7%,30.9%,0.7%,7.1%,18.6%,54.8%,52.2
7,13,Республика Мордовия,55,0.633,3.2%,59.9%,56.5%,10.4%,9.4%,,5.8%,15.3%,70.6%,45.3
8,26,Ставропольский край,48,1.141,2.5%,51.3%,52.6%,8.9%,8.2%,0.1%,5.2%,15.5%,66.8%,44.6
9,36,Воронежская область,28,1.033,3.7%,57.0%,51.9%,16.0%,13.8%,1.3%,5.5%,17.1%,59.1%,47.7


В таблице представлены:

* __school_on_10^5_people__ - количество высших учебных заведений на 100 000 человек населения
* __share_stud_people__ - отношение числа студентов к населению региона
* __share_full_stud__ - доля студентов дневной формы обучения
* __share_evening_stud__ - доля студентов вечерней фрмы обучения 
* __share_corr_stud__ - доля студентов заочной формы обучения
* __share_stud_femail__ - доля студентов женщин
* __growth_stud__ и __screening_out_stud__ - показатели соотношения количества студентов принятых на обучение и количества завершивших обучение. Не имея информации о методике сбора этих данных, их можно интерпретировать как показатель доли студентов не завершивших обучение по причине неуспеваемости, либо как показатель изменения общего количества обучающихся за время, прошедшее с года поступления студентов, завершающих образование, до времени фиксации количества поступивших студентов. Таким образом, в зависимости от выбраного метода интерпретации, признак __growth_stud__ может указывать на прирост количества обучающихся, а признак __screening_out_stud__ укажет на процент отсева студентов за время обучения. 
* __share_extern_stud__ - доля студентов, завершивших обучение экстерном
* __share_teach_stud__ - отношение количества преподавательского состава к количесту обучающихся
* __share_doc_teach__ - доля докторов наук среди преподавателей
* __share_mag_teach__ - доля кандидатов в доктора наук среди преподавателей
* __teacher_age_median__ - медианный возраст преподавательского состава.

Сортировка таблицы производится по расчетному рангу. Приоритеты для выполения расчета ранга субъектов федерации с точки зрения качества образования выбираются пользователем самостоятельно путем выбора весовых коэффициентов.

На экран выводятся строки, соответствующие условию превышения 90%-го квантиля.